In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import warnings
warnings.filterwarnings('ignore')

Parameter = {
    'ticker': 'QQQ',
    'start_date': '2014-01-01',
    'end_date': '2025-06-30',
    'output_dir': 'feature_engineering_output',
    'short_term_momentum': [2, 3, 5, 7, 10],
    'short_mid_term_trend': [14, 20, 30, 50],
    'mid_long_term_trend': [75, 100, 150, 200, 300, 500],
    'target_horizons': [5, 10, 20]
}

def get_all_time_windows():
    all_windows = (Parameter['short_term_momentum'] + 
                   Parameter['short_mid_term_trend'] + 
                   Parameter['mid_long_term_trend'])
    return sorted(all_windows)

def get_data_and_engineer_features(ticker, start_date, end_date):
    print(f"获取{ticker}数据并生成特征...")
    
    all_windows = get_all_time_windows()
    max_window = max(all_windows)
    max_horizon = max(Parameter['target_horizons'])
    
    lookback_days = int(max_window * 1.8 + 100)
    forward_days = max_horizon + 30
    
    start_dt = pd.to_datetime(start_date) - timedelta(days=lookback_days)
    end_dt = pd.to_datetime(end_date) + timedelta(days=forward_days)
    
    df = yf.Ticker(ticker).history(
        start=start_dt.strftime('%Y-%m-%d'),
        end=end_dt.strftime('%Y-%m-%d'),
        auto_adjust=False
    )
    
    if df.empty:
        raise ValueError(f"无法获取{ticker}的数据")
    
    df.index = df.index.tz_localize(None)
    df.rename(columns={
        'Open': 'open', 'High': 'high', 'Low': 'low', 
        'Close': 'close', 'Volume': 'volume'
    }, inplace=True)
    
    keep_columns = ['open', 'high', 'low', 'close', 'volume']
    df = df[keep_columns]
    
    df['turnover'] = df['close'] * df['volume']
    df['daily_return'] = df['close'].pct_change()
    
    print(f"原始数据: {len(df)}天")
    
    print("生成特征...")
    feature_count = 0
    
    for n in all_windows:
        # 1. n日收益率
        df[f'return_{n}d'] = df['close'].pct_change(periods=n)
        feature_count += 1
        
        # 2. n日平均收益
        df[f'mean_return_{n}d'] = df['daily_return'].rolling(window=n).mean()
        feature_count += 1
        
        # 3. n日波动率
        df[f'volatility_{n}d'] = df['daily_return'].rolling(window=n).std()
        feature_count += 1
        
        # 4. n日偏度 (需要>=3个数据点)
        if n >= 3:
            df[f'skewness_{n}d'] = df['daily_return'].rolling(window=n).skew()
            feature_count += 1
        
        # 5. n日峰度 (需要>=4个数据点)
        if n >= 4:
            df[f'kurtosis_{n}d'] = df['daily_return'].rolling(window=n).kurt()
            feature_count += 1
        
        # 6. n日夏普比率
        rolling_mean = df['daily_return'].rolling(window=n).mean()
        rolling_std = df['daily_return'].rolling(window=n).std()
        df[f'sharpe_{n}d'] = rolling_mean / rolling_std
        feature_count += 1
    
    print(f"生成{feature_count}个特征")
    
    end_date_dt = pd.to_datetime(end_date)
    result_df = df[(df.index >= pd.to_datetime(start_date)) & (df.index <= end_date_dt)].copy()
    
    print(f"最终数据集: {len(result_df)}天, {len(result_df.columns)}列")
    
    return result_df

def generate_target_variables(df):
    print("生成目标变量...")
    
    target_count = 0
    
    for horizon in Parameter['target_horizons']:
        # 1. 未来收益率
        df[f'y_ret_{horizon}d'] = df['close'].pct_change(periods=horizon).shift(-horizon)
        target_count += 1
        
        # 2. 未来夏普比率 (仅10日和20日)
        if horizon >= 10:
            future_returns = df['daily_return'].shift(-1)
            future_mean = future_returns.rolling(window=horizon).mean().shift(-(horizon-1))
            future_std = future_returns.rolling(window=horizon).std().shift(-(horizon-1))
            df[f'y_sharpe_{horizon}d'] = (future_mean * np.sqrt(252)) / (future_std * np.sqrt(252))
            target_count += 1
    
    print(f"生成{target_count}个目标变量")
    return df

def extract_features_and_targets(df):
    print("提取特征X和目标y...")
    
    # 排除的列
    exclude_cols = ['open', 'high', 'low', 'close', 'volume', 'turnover', 'daily_return']
    
    # 目标列
    target_cols = [col for col in df.columns if col.startswith('y_')]
    
    # 特征列
    feature_cols = [col for col in df.columns if col not in exclude_cols + target_cols]
    
    print(f"特征数量: {len(feature_cols)}")
    print(f"目标数量: {len(target_cols)}")
    print(f"目标变量: {target_cols}")
    
    # 提取数据
    X = df[feature_cols]
    y = df[target_cols]
    
    # 删除有缺失值的行
    complete_mask = ~(X.isnull().any(axis=1) | y.isnull().any(axis=1))
    X_clean = X[complete_mask]
    y_clean = y[complete_mask]
    
    print(f"删除缺失值后: {len(X_clean)}行")
    print(f"数据完整性: {len(X_clean)/len(df)*100:.1f}%")
    
    return X_clean, y_clean, feature_cols, target_cols

def export_to_excel(df, X, y, feature_names, target_names):
    print("导出数据到Excel...")
    
    if not os.path.exists(Parameter['output_dir']):
        os.makedirs(Parameter['output_dir'])
    
    import time
    timestamp = int(time.time())
    output_path = os.path.join(Parameter['output_dir'], f'QQQ_ML_dataset_{timestamp}.xlsx')
    
    export_df = df.copy()
    export_df.reset_index(inplace=True)
    
    chinese_columns = {
        'Date': '日期',
        'open': '开盘价',
        'high': '最高价', 
        'low': '最低价',
        'close': '收盘价',
        'volume': '成交量',
        'turnover': '交易额',
        'daily_return': '日收益率'
    }
    
    for col in export_df.columns:
        if col in chinese_columns:
            continue
        elif col.startswith('return_'):
            days = col.replace('return_', '').replace('d', '')
            chinese_columns[col] = f'{days}日收益率'
        elif col.startswith('mean_return_'):
            days = col.replace('mean_return_', '').replace('d', '')
            chinese_columns[col] = f'{days}日平均收益'
        elif col.startswith('volatility_'):
            days = col.replace('volatility_', '').replace('d', '')
            chinese_columns[col] = f'{days}日波动率'
        elif col.startswith('skewness_'):
            days = col.replace('skewness_', '').replace('d', '')
            chinese_columns[col] = f'{days}日偏度'
        elif col.startswith('kurtosis_'):
            days = col.replace('kurtosis_', '').replace('d', '')
            chinese_columns[col] = f'{days}日峰度'
        elif col.startswith('sharpe_'):
            days = col.replace('sharpe_', '').replace('d', '')
            chinese_columns[col] = f'{days}日夏普比率'
        elif col.startswith('y_ret_'):
            days = col.replace('y_ret_', '').replace('d', '')
            chinese_columns[col] = f'目标_未来{days}日回报率'
        elif col.startswith('y_sharpe_'):
            days = col.replace('y_sharpe_', '').replace('d', '')
            chinese_columns[col] = f'目标_未来{days}日夏普比率'
    
    export_df.rename(columns=chinese_columns, inplace=True)
    export_df['日期'] = export_df['日期'].dt.strftime('%Y-%m-%d')
    
    # 创建项目概览
    overview_data = {
        '项目信息': [
            '目标标的', '开始日期', '结束日期', '总交易日数',
            '特征数量', '目标数量', '数据完整率(%)', '文件生成时间'
        ],
        '具体值': [
            Parameter['ticker'], Parameter['start_date'], Parameter['end_date'], 
            len(export_df), len(feature_names), len(target_names),
            f"{len(X)/len(df)*100:.1f}", datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        ]
    }
    overview_df = pd.DataFrame(overview_data)
    
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        # 主数据
        export_df.to_excel(writer, sheet_name='完整数据集', index=False)
        
        # 项目概览
        overview_df.to_excel(writer, sheet_name='项目概览', index=False)
        
        # 特征列表
        feature_df = pd.DataFrame({
            '特征名称': feature_names,
            '中文名称': [chinese_columns.get(f, f) for f in feature_names]
        })
        feature_df.to_excel(writer, sheet_name='特征列表', index=False)
        
        # 目标列表
        target_df = pd.DataFrame({
            '目标名称': target_names,
            '中文名称': [chinese_columns.get(t, t) for t in target_names]
        })
        target_df.to_excel(writer, sheet_name='目标列表', index=False)
    
    print(f"数据已导出: {output_path}")
    return output_path
    
    print("提取特征X和目标y...")
    
    exclude_cols = ['open', 'high', 'low', 'close', 'volume', 'turnover', 'daily_return']
    
    target_cols = [col for col in df.columns if col.startswith('y_')]
    
    feature_cols = [col for col in df.columns if col not in exclude_cols + target_cols]
    
    print(f"特征数量: {len(feature_cols)}")
    print(f"目标数量: {len(target_cols)}")
    print(f"目标变量: {target_cols}")
    
    X = df[feature_cols]
    y = df[target_cols]
    
    complete_mask = ~(X.isnull().any(axis=1) | y.isnull().any(axis=1))
    X_clean = X[complete_mask]
    y_clean = y[complete_mask]
    
    print(f"删除缺失值后: {len(X_clean)}行")
    print(f"数据完整性: {len(X_clean)/len(df)*100:.1f}%")
    
    return X_clean, y_clean, feature_cols, target_cols

def main():
    print("QQQ特征工程系统")
    print("="*40)
    
    try:
        # 1. 获取数据并生成特征
        df_with_features = get_data_and_engineer_features(
            Parameter['ticker'], 
            Parameter['start_date'], 
            Parameter['end_date']
        )
        
        # 2. 生成目标变量
        df_with_targets = generate_target_variables(df_with_features)
        
        # 3. 提取X和y
        X, y, feature_names, target_names = extract_features_and_targets(df_with_targets)
        
        # 4. 导出Excel
        output_path = export_to_excel(df_with_targets, X, y, feature_names, target_names)
        
        print("\n" + "="*40)
        print("特征工程完成!")
        print(f"特征矩阵X: {X.shape}")
        print(f"目标矩阵y: {y.shape}")
        print(f"Excel文件: {output_path}")
        print("="*40)
        
        # 返回结果
        return X, y, feature_names, target_names, df_with_targets, output_path
        
    except Exception as e:
        print(f"错误: {e}")
        import traceback
        traceback.print_exc()
        return None, None, None, None, None, None

if __name__ == '__main__':
    X, y, feature_names, target_names, full_data, excel_path = main()
    
    if X is not None:
        print(f"\n使用示例:")
        print(f"X.shape: {X.shape}")
        print(f"y.shape: {y.shape}")
        print(f"特征名称: feature_names[:5] = {feature_names[:5]}")
        print(f"目标名称: target_names = {target_names}")
        print(f"完整数据: full_data.shape = {full_data.shape}")
        print(f"Excel文件: {excel_path}")

QQQ特征工程系统
获取QQQ数据并生成特征...
原始数据: 3594天
生成特征...
生成87个特征
最终数据集: 2890天, 94列
生成目标变量...
生成5个目标变量
提取特征X和目标y...
特征数量: 87
目标数量: 5
目标变量: ['y_ret_5d', 'y_ret_10d', 'y_sharpe_10d', 'y_ret_20d', 'y_sharpe_20d']
删除缺失值后: 2870行
数据完整性: 99.3%
导出数据到Excel...
数据已导出: feature_engineering_output\QQQ_ML_dataset_1753341108.xlsx

特征工程完成!
特征矩阵X: (2870, 87)
目标矩阵y: (2870, 5)
Excel文件: feature_engineering_output\QQQ_ML_dataset_1753341108.xlsx

使用示例:
X.shape: (2870, 87)
y.shape: (2870, 5)
特征名称: feature_names[:5] = ['return_2d', 'mean_return_2d', 'volatility_2d', 'sharpe_2d', 'return_3d']
目标名称: target_names = ['y_ret_5d', 'y_ret_10d', 'y_sharpe_10d', 'y_ret_20d', 'y_sharpe_20d']
完整数据: full_data.shape = (2890, 99)
Excel文件: feature_engineering_output\QQQ_ML_dataset_1753341108.xlsx
